<a href="https://colab.research.google.com/github/theostoican/Guided-Research/blob/main/gr_vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoFeatureExtractor, ViTMAEForPreTraining
from PIL import Image
import requests
import os
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt


url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)
image = Image.open('frame000001.jpg')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 26.8 MB/s 
     |████████████████████████████████| 101 kB 11.0 MB/s 
     |████████████████████████████████| 6.6 MB 30.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-384')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-384', output_attentions=True)

# feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/vit-mae-base', output_attentions=True)
# model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')

inputs = feature_extractor(images=image, return_tensors="pt")
print(inputs.pixel_values.shape)

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

torch.Size([1, 3, 384, 384])


In [ ]:
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: rugby ball


In [ ]:
outputs = model(**inputs)

In [ ]:
print(len(outputs.attentions))

12


In [ ]:
print(outputs.attentions[-1].shape)

torch.Size([1, 12, 577, 577])


In [ ]:
attentions = outputs.attentions[-1].detach()

nh = attentions.shape[1] # number of heads


patch_size = 16
print (inputs.pixel_values.shape)
w, h = inputs.pixel_values.shape[-2] - inputs.pixel_values.shape[-2] % patch_size, inputs.pixel_values.shape[-1] - inputs.pixel_values.shape[-1] % patch_size
inputs.pixel_values = inputs.pixel_values[:, :w, :h].unsqueeze(0)

w_featmap = inputs.pixel_values.shape[-2] // patch_size
h_featmap = inputs.pixel_values.shape[-1] // patch_size

attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

attentions = attentions.reshape(nh, w_featmap, h_featmap)
attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0].cpu().numpy()

# save attentions heatmaps
os.makedirs('maps', exist_ok=True)
torchvision.utils.save_image(torchvision.utils.make_grid(inputs.pixel_values, normalize=True, scale_each=True),
                             os.path.join('maps', "img.png"))
for j in range(nh):
    fname = os.path.join('maps', "attn-head" + str(j) + ".png")
    plt.imsave(fname=fname, arr=attentions[j], format='png')
    print(f"{fname} saved.")

torch.Size([1, 3, 384, 384])
maps/attn-head0.png saved.
maps/attn-head1.png saved.
maps/attn-head2.png saved.
maps/attn-head3.png saved.
maps/attn-head4.png saved.
maps/attn-head5.png saved.
maps/attn-head6.png saved.
maps/attn-head7.png saved.
maps/attn-head8.png saved.
maps/attn-head9.png saved.
maps/attn-head10.png saved.
maps/attn-head11.png saved.
